In [1]:
!pip install ydata-synthetic==1.1.0


In [2]:
from ydata_synthetic.synthesizers.regular import RegularSynthesizer
from ydata_synthetic.synthesizers import ModelParameters, TrainParameters
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#Load data and define the data processor parameters
df = pd.read_csv('https://raw.githubusercontent.com/mahayasa/gan-hybrid-sampling-customer-churn/main/data/telco_new.csv')


df_numerical_features = df.select_dtypes(exclude='object')
df_categorical_features = df.select_dtypes(include='object')

num_cols = list(df_numerical_features)
cat_cols = list(df_categorical_features)


In [3]:
train_data = df.loc[ df['Churn']==1 ].copy()

train_data

,Call Failure,Complains,Subscription Length,Charge Amount,Seconds of Use,Frequency of use,Frequency of SMS,Distinct Called Numbers,Age Group,Tariff Plan,Status,Age,Customer Value,FN,FP,Churn
16,0,0,37,0,875,14,0,11,2,1,2,25,40.005,36.0045,60.0000,1
18,0,0,37,0,0,0,0,0,2,1,2,25,0.000,0.0000,60.0000,1
22,23,1,33,0,955,47,16,17,2,1,2,25,117.090,105.3810,61.7090,1
24,13,1,36,1,5818,98,26,24,2,1,1,25,383.220,344.8980,88.3220,1
26,9,0,35,0,2990,41,9,16,3,1,2,30,157.240,141.5160,65.7240,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3126,14,1,37,0,3295,47,16,18,3,1,2,30,197.680,177.9120,69.7680,1
3127,14,1,38,0,2573,50,41,33,4,1,2,45,168.075,151.2675,66.8075,1
3128,5,0,38,0,438,8,7,4,3,1,2,30,45.840,41.2560,60.0000,1
3131,5,1,38,0,933,13,16,6,4,1,2,45,63.650,57.2850,60.0000,1


In [4]:
#Defining the training parameters
noise_dim = 128
dim = 128
batch_size = 500

log_step = 100
epochs = 100
learning_rate = [5e-4, 3e-3]
beta_1 = 0.5
beta_2 = 0.9
models_dir = '../cache'

gan_args = ModelParameters(batch_size=batch_size,
                           lr=learning_rate,
                           betas=(beta_1, beta_2),
                           noise_dim=noise_dim,
                           layers_dim=dim)

train_args = TrainParameters(epochs=epochs,
                             sample_interval=log_step)

synth = RegularSynthesizer(modelname='wgangp', model_parameters=gan_args, n_critic=2)
synth.fit(train_data, train_args, num_cols, cat_cols)

synth.save('telco3_wgangp_model.pkl')



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
  1%|          | 1/100 [00:07<12:01,  7.29s/it]

Epoch: 0 | disc_loss: 235.69412231445312 | gen_loss: 0.015346145257353783


  2%|▏         | 2/100 [00:07<05:14,  3.21s/it]

Epoch: 1 | disc_loss: 21.053491592407227 | gen_loss: -0.0026631138753145933


  3%|▎         | 3/100 [00:07<03:02,  1.88s/it]

...

Epoch: 97 | disc_loss: 0.37024396657943726 | gen_loss: -0.0742790549993515


 99%|█████████▉| 99/100 [00:38<00:00,  3.40it/s]

Epoch: 98 | disc_loss: 1.469687581062317 | gen_loss: -0.07522572576999664


100%|██████████| 100/100 [00:38<00:00,  2.61it/s]

Epoch: 99 | disc_loss: 0.006346283946186304 | gen_loss: -0.0744447261095047


In [5]:
#########################################################
#    Loading and sampling from a trained synthesizer    #
#########################################################
synth = RegularSynthesizer.load('telco3_wgangp_model.pkl')
synth_data = synth.sample(2655)

Synthetic data generation: 100%|██████████| 6/6 [00:00<00:00, 38.18it/s]


In [6]:
synth_data[synth_data['Churn']==1]

,Call Failure,Complains,Subscription Length,Charge Amount,Seconds of Use,Frequency of use,Frequency of SMS,Distinct Called Numbers,Age Group,Tariff Plan,Status,Age,Customer Value,FN,FP,Churn
1,2,0,12,0,1363,31,13,-8,2,1,1,30,-70.516228,132.506332,78.000946,1
3,1,0,6,0,1203,12,-4,-11,2,1,1,26,-27.763872,44.404381,62.174450,1
4,2,0,11,0,1536,23,-2,-12,2,1,1,28,-88.418213,134.565491,62.299126,1
6,0,0,7,0,1891,21,-2,-6,2,1,1,26,28.114433,78.191055,70.817413,1
11,1,0,12,0,1639,26,-2,-9,2,1,1,28,54.852772,171.096161,71.902992,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2986,1,0,9,0,760,22,14,-13,2,1,1,30,-8.575007,122.128227,73.974800,1
2991,0,0,7,0,1229,19,-10,-10,2,1,1,27,27.851370,67.610306,67.017921,1
2992,0,0,7,0,1131,13,15,-11,2,1,1,28,-4.924858,131.477890,75.763634,1
2994,2,0,7,0,1086,31,5,-12,2,1,1,28,-50.627995,124.854027,69.083656,1


In [7]:
#concat original data and gan data
data_concat = pd.concat([df, synth_data])
# combine data churn and not churn
data=pd.concat([df, data_concat])

In [ ]:
data

,Call Failure,Complains,Subscription Length,Charge Amount,Seconds of Use,Frequency of use,Frequency of SMS,Distinct Called Numbers,Age Group,Tariff Plan,Status,Age,Customer Value,FN,FP,Churn
0,8,0,38,0,4370,71,5,17,3,1,1,30,197.640000,177.876000,69.764000,0
1,0,0,39,0,318,5,7,4,2,1,2,25,46.035000,41.431500,60.000000,0
2,10,0,37,0,2453,60,359,24,3,1,1,30,1536.520000,1382.868000,203.652000,0
3,10,0,38,0,4198,66,1,35,1,1,1,15,240.020000,216.018000,74.002000,0
4,3,0,38,0,2393,58,2,33,1,1,1,15,145.805000,131.224500,64.580500,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,1,0,29,0,-1541,10,56,6,1,0,1,34,10.451343,150.171158,61.966034,1
2996,2,0,30,0,-1254,12,55,4,1,0,1,36,-41.251778,183.216614,75.993462,1
2997,0,0,32,0,-1019,7,61,8,1,0,1,33,-82.881813,192.054489,67.483727,1
2998,0,0,32,0,-1329,20,65,12,1,0,1,30,8.565728,226.877243,77.854240,1


In [8]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun  1 17:31:40 2020

@author: manav

Modifed on 23 AUG 2022

by mahayasa adiputra
"""

import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
import sklearn.metrics as mt
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from numpy import mean
from numpy import std
from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
import time

start1=time.time()

X=data.drop(['Churn'],axis=1)
y=data["Churn"]

enn = EditedNearestNeighbours(n_neighbors=3)
X, y = enn.fit_resample(X, y)
#ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
#X, y = ncr.fit_resample(X, y)
#tomek_links = TomekLinks()
#X, y = tomek_links.fit_resample(X, y)

#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)
end1 = time.time()
print("The time of execution of preprocess:",
      (end1-start1), "s")

The time of execution of preprocess: 0.18804717063903809 s


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
# decision tree
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangpenn-telco3-dt-cs-auc.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/wgangpenn-telco3-dt-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.855329 using {'class_weight': {0: 40, 1: 100}}
0.851795 (0.013961) with: {'class_weight': {0: 10, 1: 100}}
0.850942 (0.014248) with: {'class_weight': {0: 20, 1: 100}}
0.848781 (0.012169) with: {'class_weight': {0: 30, 1: 100}}
0.855329 (0.013858) with: {'class_weight': {0: 40, 1: 100}}
0.850366 (0.009448) with: {'class_weight': {0: 50, 1: 100}}
0.851660 (0.013979) with: {'class_weight': {0: 60, 1: 100}}
0.852265 (0.010349) with: {'class_weight': {0: 70, 1: 100}}
0.852125 (0.011502) with: {'class_weight': {0: 80, 1: 100}}
0.851652 (0.014484) with: {'class_weight': {0: 90, 1: 100}}
0.851391 (0.013424) with: {'class_weight': {0: 100, 1: 100}}


In [20]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangpenn-telco3-dt-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/wgangpenn-telco3-dt-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.856526 using {'class_weight': {0: 100, 1: 100}}
0.855266 (0.010079) with: {'class_weight': {0: 10, 1: 100}}
0.853855 (0.010117) with: {'class_weight': {0: 20, 1: 100}}
0.851015 (0.011521) with: {'class_weight': {0: 30, 1: 100}}
0.855869 (0.008566) with: {'class_weight': {0: 40, 1: 100}}
0.852682 (0.010035) with: {'class_weight': {0: 50, 1: 100}}
0.854271 (0.010079) with: {'class_weight': {0: 60, 1: 100}}
0.854428 (0.009002) with: {'class_weight': {0: 70, 1: 100}}
0.851840 (0.012493) with: {'class_weight': {0: 80, 1: 100}}
0.853210 (0.011400) with: {'class_weight': {0: 90, 1: 100}}
0.856526 (0.010370) with: {'class_weight': {0: 100, 1: 100}}


In [21]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangpenn-telco3-dt-cs-gmean.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/wgangpenn-telco3-dt-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.851055 using {'class_weight': {0: 100, 1: 100}}
0.846447 (0.014160) with: {'class_weight': {0: 10, 1: 100}}
0.842601 (0.013513) with: {'class_weight': {0: 20, 1: 100}}
0.843110 (0.014275) with: {'class_weight': {0: 30, 1: 100}}
0.850603 (0.013898) with: {'class_weight': {0: 40, 1: 100}}
0.844079 (0.013283) with: {'class_weight': {0: 50, 1: 100}}
0.846327 (0.017192) with: {'class_weight': {0: 60, 1: 100}}
0.848838 (0.009449) with: {'class_weight': {0: 70, 1: 100}}
0.848414 (0.015133) with: {'class_weight': {0: 80, 1: 100}}
0.847632 (0.014612) with: {'class_weight': {0: 90, 1: 100}}
0.851055 (0.015437) with: {'class_weight': {0: 100, 1: 100}}


In [22]:
# LR
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangpenn-telco3-lr-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/wgangpenn-telco3-lr-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.829606 using {'class_weight': {0: 60, 1: 100}}
0.827220 (0.011296) with: {'class_weight': {0: 10, 1: 100}}
0.827991 (0.011780) with: {'class_weight': {0: 20, 1: 100}}
0.829044 (0.011131) with: {'class_weight': {0: 30, 1: 100}}
0.828122 (0.012236) with: {'class_weight': {0: 40, 1: 100}}
0.828798 (0.011369) with: {'class_weight': {0: 50, 1: 100}}
0.829606 (0.011725) with: {'class_weight': {0: 60, 1: 100}}
0.828731 (0.012158) with: {'class_weight': {0: 70, 1: 100}}
0.828843 (0.012437) with: {'class_weight': {0: 80, 1: 100}}
0.829140 (0.011702) with: {'class_weight': {0: 90, 1: 100}}
0.829061 (0.011904) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangpenn-telco3-lr-cs-f1.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/wgangpenn-telco3-lr-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.693068 using {'class_weight': {0: 50, 1: 100}}
0.611600 (0.010727) with: {'class_weight': {0: 10, 1: 100}}
0.660506 (0.012006) with: {'class_weight': {0: 20, 1: 100}}
0.681976 (0.009369) with: {'class_weight': {0: 30, 1: 100}}
0.688484 (0.011669) with: {'class_weight': {0: 40, 1: 100}}
0.693068 (0.014012) with: {'class_weight': {0: 50, 1: 100}}
0.690846 (0.011758) with: {'class_weight': {0: 60, 1: 100}}
0.673089 (0.020064) with: {'class_weight': {0: 70, 1: 100}}
0.655571 (0.018426) with: {'class_weight': {0: 80, 1: 100}}
0.645208 (0.017467) with: {'class_weight': {0: 90, 1: 100}}
0.635250 (0.017497) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangpenn-telco3-lr-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/wgangpenn-telco3-lr-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.763447 using {'class_weight': {0: 30, 1: 100}}
0.712994 (0.010873) with: {'class_weight': {0: 10, 1: 100}}
0.753518 (0.011980) with: {'class_weight': {0: 20, 1: 100}}
0.763447 (0.010737) with: {'class_weight': {0: 30, 1: 100}}
0.754767 (0.013216) with: {'class_weight': {0: 40, 1: 100}}
0.737841 (0.018174) with: {'class_weight': {0: 50, 1: 100}}
0.707749 (0.014811) with: {'class_weight': {0: 60, 1: 100}}
0.653222 (0.027631) with: {'class_weight': {0: 70, 1: 100}}
0.600379 (0.027865) with: {'class_weight': {0: 80, 1: 100}}
0.561258 (0.025866) with: {'class_weight': {0: 90, 1: 100}}
0.529908 (0.027119) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [25]:
#SVM
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
model = SVC()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangpenn-telco3-svm-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/wgangpenn-telco3-svm-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.816485 using {'class_weight': {0: 50, 1: 100}}
0.726224 (0.010446) with: {'class_weight': {0: 10, 1: 100}}
0.805121 (0.010652) with: {'class_weight': {0: 20, 1: 100}}
0.810107 (0.012210) with: {'class_weight': {0: 30, 1: 100}}
0.813736 (0.012372) with: {'class_weight': {0: 40, 1: 100}}
0.816485 (0.012574) with: {'class_weight': {0: 50, 1: 100}}
0.812657 (0.012618) with: {'class_weight': {0: 60, 1: 100}}
0.813975 (0.012870) with: {'class_weight': {0: 70, 1: 100}}
0.816370 (0.014149) with: {'class_weight': {0: 80, 1: 100}}
0.812522 (0.014490) with: {'class_weight': {0: 90, 1: 100}}
0.811790 (0.019260) with: {'class_weight': {0: 100, 1: 100}}


In [26]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangpenn-telco3-svm-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/wgangpenn-telco3-svm-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.673698 using {'class_weight': {0: 50, 1: 100}}
0.542874 (0.010562) with: {'class_weight': {0: 10, 1: 100}}
0.623016 (0.011221) with: {'class_weight': {0: 20, 1: 100}}
0.635611 (0.012121) with: {'class_weight': {0: 30, 1: 100}}
0.645897 (0.011644) with: {'class_weight': {0: 40, 1: 100}}
0.673698 (0.011816) with: {'class_weight': {0: 50, 1: 100}}
0.670269 (0.011751) with: {'class_weight': {0: 60, 1: 100}}
0.631272 (0.043458) with: {'class_weight': {0: 70, 1: 100}}
0.563937 (0.015445) with: {'class_weight': {0: 80, 1: 100}}
0.563752 (0.015457) with: {'class_weight': {0: 90, 1: 100}}
0.549104 (0.011325) with: {'class_weight': {0: 100, 1: 100}}


In [27]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('wgangpenn-telco3-svm-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/wgangpenn-telco3-svm-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco3"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.738246 using {'class_weight': {0: 50, 1: 100}}
0.645098 (0.011548) with: {'class_weight': {0: 10, 1: 100}}
0.722662 (0.011602) with: {'class_weight': {0: 20, 1: 100}}
0.729282 (0.012789) with: {'class_weight': {0: 30, 1: 100}}
0.725768 (0.011861) with: {'class_weight': {0: 40, 1: 100}}
0.738246 (0.012790) with: {'class_weight': {0: 50, 1: 100}}
0.682344 (0.013466) with: {'class_weight': {0: 60, 1: 100}}
0.547905 (0.095416) with: {'class_weight': {0: 70, 1: 100}}
0.391078 (0.024604) with: {'class_weight': {0: 80, 1: 100}}
0.389888 (0.024277) with: {'class_weight': {0: 90, 1: 100}}
0.359679 (0.023230) with: {'class_weight': {0: 100, 1: 100}}
